## Implementation of Variable Elimination to approximate probability queries from a Bayesian Network.

In [11]:
# Given an arbitrary Bayesian Network, an arbitrary set of query variables, and arbitrary evidence, we want to be able to calculate the probability of said variable...
import numpy as np

def join(first: np.array, second: np.array, row_pairings: dict[int,list[int]]) -> np.array:
    """Given two distributions and the variable to to join on, join the two distributions into a third

    Args:
        first (np.ndarray): first distribution (smaller or same size)
        second (np.ndarray): second distribution (larger or same size)
        row_pairings (dict[int,list[int]]): map of rows of smaller first distribution mapping to rows of second distribution they will multiply

    Returns:
        np.ndarray: resulting distribution after joining
    """
    result = np.zeros(second.shape)
    for first_index, second_indices in row_pairings.items():
        for s in second_indices:
            result[s] += first[first_index] * second[s]
    return result

def eliminate(distribution: np.array, pairings: list[tuple[int,int]]) -> np.array:
    """eliminate a variable by summing over the distribution at the specified index pairs

    Args:
        distribution (np.ndarray): pairs of indices we add together

    Returns:
        np.ndarray: resulting distribution (dimensions will be half as large)
    """
    result = np.array(shape=len(pairings))
    for i, pair in enumerate(pairings):
        result[i] = distribution[pair[0]] + distribution[pair[1]]
    return result

In [4]:
distr_1 = np.array([.000999, .00029, .93906, .00095])
distr_2 = np.array([.95,.05,.55,.45,.17,.83,.23,.77])
joined = join(distr_1, distr_2, {0:[0,1],1:[2,3],2:[4,5],3:[6,7]})

In [12]:
eliminate(joined, pairings=[(0,2),(1,3),(4,6),(5,7)])

array([1.108550e-03, 1.804500e-04, 1.598587e-01, 7.801513e-01])

In [ ]:
def eliminate_variable(var: int, relevant_factors: list[tuple[list[int],np.array]]) -> tuple[list[int],np.array]:
    """Given which variable we want to eliminate, multiply its respective factors together and then sum out over the variable to eliminate

    Args:
        var (int): the variable to eliminate
        factors (list[tuple[list[int],np.array]]): list of distributions corresponding to this variable

    Returns:
        tuple[list[int],np.array]: new list of relevant variables to this array along with the array (a new distribution)
    """
    # We need to find out which rows multiply together
    # Break into pairs
    # TODO - revise approach and complete
    prev_factor = relevant_factors[0]
    for i in range(1, len(relevant_factors)):
        next_factor = relevant_factors[i]
        merged = None
        # TODO - continue
        prev_factor = merged

def relevant(parents: list[int], parents_in_evidence: list[int], evidence: dict[int,bool], row: int) -> bool:
    """Based on the parents' order for a given probability array, which parents are in the evidence, and whether those parents are true or false, determine if the row is consistent with the evidence based on our probability array construction convention

    Args:
        parents (list[int]): list of parents in order for some probability array
        parents_in_evidence (list[int]): list of parents which are in the evidence
        evidence (dict[int,bool]): records which variables in the evidence are true or false
        row (int): row index to determine if consistent with evidence

    Returns:
        bool: whether row number is consistent with evidence
    """
    for parent in parents_in_evidence:
        parent_idx = parents.index(parent)
        binary_posn = len(parents) - 1 - parent_idx
        switch_every = 2 ** binary_posn
        negative_row = ((row // switch_every) % 2) == 0
        if (negative_row and evidence[parent]) or ((not negative_row) and (not evidence[parent])):
            return False
    return True

def return_query_probabilities(queries: list[int], evidence: dict[int,bool], network: dict) -> np.array:
    """Given a Bayesian network and a list of query and evidence variables, return the probability distribution for all possible values of query variables

    Args:
        queries (list[int]): list of variables specified whose value probabilities we want to query
        evidence (list[tuple[int,bool]]): list of variables whose values are specified and hence affect query probabilities
        network (dict): underlying network which reveals probabilities of each node given its parents' values

    Returns:
        np.ndarray: probability distribution of possible combination values of each of the query variables (2^{#query variables}, 0 is all false and 2^{#query variables}-1 is all true)
    """
    # we need a map of variables to their respective factors (some of which will be shared) - factors are integers serving as references to numpy arrays
    factor_mappings = {i:[] for i in network.keys()}
    factors = {}
    # each factor has its own probability distribution - which may or may not depend on its parents
    for i, info in network.items():
        # if i is not in evidence, we have work to do
        if i in evidence.keys():
            continue

        parents = info["parents"]
        parents_in_evidence = [p for p in parents if p in evidence.keys()]
        parents_not_in_evidence = [p for p in parents if p not in evidence.keys()]
        # obtain the array of probabilities before considering evidence
        prob_array = np.array([pair[1] for pair in info["prob"]])

        # now filter the probability array so that it only contains entries consistent with the evidence
        relevant_rows = []
        for row in range(len(prob_array)):
            if relevant(parents, parents_in_evidence, evidence, row):
                relevant_rows.append(row)
        # quick sanity check
        assert len(relevant_rows) == 2 ** len(parents_not_in_evidence)

        # our base array size depends on the number of parents not in our evidence (and so take on both true and false values)
        new_prob_array = np.zeros(len(relevant_rows))
        # fill new_prob_array with relevant rows
        for i, row in enumerate(relevant_rows):
            new_prob_array[i] = prob_array[row]

        # double the size of our array so that it can contain the probabilities of 'i' being true and false
        final_prob_array = np.zeros(2 * len(new_prob_array))
        for i in range(len(prob_array)):
            final_prob_array[2*i] = 1 - new_prob_array[i]
            final_prob_array[2*i+1] = new_prob_array[i]

        # finally store into a tuple
        prob_tuple = ([parent for parent in parents] + [i], final_prob_array)
        factors[len(factors)] = prob_tuple
        # each involved variable needs THAT reference to prob_tuple
        factor_mappings[i].append(len(factors))
        for p in parents:
            factor_mappings[p].append(len(factors))
    
    # figure out which variables need to be eliminated
    query_set = set(queries)
    evidence_set = set([pair[0] for pair in evidence])
    hidden_vars = [i for i in range(len(network)) if i not in query_set and i not in evidence_set]
    # eliminate the hidden variables in an order based on their number of factors - more factors means eliminate sooner
    hidden_vars.sort(key=lambda i : -len(factor_mappings[i]))


In [14]:
a = np.array([1.0])
lst = [a]
print(lst)
a = None
print(lst)

[array([1.])]
[array([1.])]
